In [138]:
# loading libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
import xgboost as xgb
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV as GridSearch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve,roc_curve


In [139]:
features_gaby = pd.read_csv('../features/features_gaby.csv')
features_gonza = pd.read_csv('../features/features_gonza.csv').drop(columns='Unnamed: 0')
features_ari = pd.read_csv('../features/features_ariel.csv')
features_mati = pd.read_csv('../features/features_frecuencia.csv')
labels_training_set = pd.read_csv('../dataset/labels_training_set.csv')
test_set = pd.read_csv('../dataset/trocafone_kaggle_test.csv')
features_gonza2 = pd.read_csv('../features/features_gonza2.csv')

In [140]:
#model_df = pd.merge(features_gaby, features_gonza, how='inner', on='person')
model_df = pd.merge(features_gaby, features_ari, how='inner', on='person')
model_df = pd.merge(model_df, features_gonza, how='inner', on='person')
model_df = pd.merge(model_df, features_gonza2, how='inner', on='person')
#model_df = pd.merge(model_df, features_mati, how='inner', on='person')
model_df.head()

person  marca_favorita_apple  marca_favorita_asus  marca_favorita_lenovo  \
0  0008ed71                     1                    0                      0   
1  00091926                     1                    0                      0   
2  00091a7a                     1                    0                      0   
3  000ba417                     0                    0                      0   
4  000c79fe                     1                    0                      0   

   marca_favorita_lg  marca_favorita_motorola  marca_favorita_no tiene  \
0                  0                        0                        0   
1                  0                        0                        0   
2                  0                        0                        0   
3                  0                        0                        0   
4                  0                        0                        0   

   marca_favorita_quantum  marca_favorita_samsung  marca_favorita_sony  \
0                       0                       0                    0   
1                       0                       0                    0   
2                       0                       0                    0   
3                       0                       1                    0   
4                       0                       0                    0   

            ...             checkout  generic listing  searched products  \
0           ...                  3.0              1.0                NaN   
1           ...                  2.0              NaN                NaN   
2           ...                  NaN              NaN                NaN   
3           ...                  6.0             14.0                NaN   
4           ...                  1.0              1.0                9.0   

   viewed product  visited site  completa_flujo  favorite_device_Computer  \
0             NaN           2.0           False                         1   
1           372.0          34.0           False                         1   
2             3.0           1.0           False                         0   
3           153.0           6.0           False                         1   
4             3.0           1.0           False                         0   

   favorite_device_Smartphone  favorite_device_Tablet  favorite_device_Unknown  
0                           0                       0                        0  
1                           0                       0                        0  
2                           1                       0                        0  
3                           0                       0                        0  
4                           1                       0                        0  

[5 rows x 64 columns]

In [141]:
#model_df.fillna(value=0, inplace=True)
model_df['most_viewed_sku'] = model_df['most_viewed_sku'].astype('category')
model_df['favorite_device_Computer'] = model_df['favorite_device_Computer'].astype('category')
model_df['viewed product'] = model_df['viewed product'].astype('category')
training_set = pd.merge(model_df, labels_training_set, how='inner', on='person')
test_set = pd.merge(model_df, test_set, how='inner', on='person')
#training_set.drop(columns= 'Unnamed: 0', inplace=True)

In [142]:
features = ['cant_total_eventos', #f1
            #'marca_favorita_apple',#f2
            'mean_events_per_day',#f3
            #'has_converted_before',#f4
            #'most_viewed_sku',#f5
            #'most_viewed_color_Preto',#f6
            #'most_viewed_color_Branco',#f7
            #'most_viewed_color_Dourado', #f8
            #'region_brasil_Sao Paulo',#f9
            #'region_brasil_Unknown',#f10
            #'region_brasil_Minas Gerais',#f11
            #'marca_favorita_samsung',#f12
            #'cant_ev_noche',#f13
            #'cant_ev_manana',#f14
            #'cant_ev_tarde',#f15
            #'cant_ev_madrugada',#f16
            #'cant_buenos',#f17
            #'cant_muy_buenos',#f18
            #'cant_excelentes',#f19
            #'nivel_de_actividad_Muy Bajo',#f20
            #'nivel_de_actividad_Muy Alto',#f21
            #'nivel_de_actividad_Bajo',#f22
            #'nivel_de_actividad_Medio',#f23
            #'nivel_de_actividad_Alto',#f24
            #'favorite_device_Smartphone',#f25
            'favorite_device_Computer',#f26
            'cantidad_checkouts',#f27
            'cantidad_conversions',#f28
            'tasa_cc',
            'viewed product',
            #'cantidad_lead',
            #'searched products',
            #'first_and_last_event_distance_in_days',
            'month_moda',
            #'completa_flujo'
           ]

In [143]:
x = np.array(training_set[features])
y = np.array(training_set['label'])
z = np.array(test_set[features])
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20)

In [144]:
def modelfit(alg):
    alg.fit(X_train, y_train, eval_metric='aucpr')
    pred = (alg.predict_proba(X_test)[:,1])
    accuracy = accuracy_score(y_test, pred.round(), normalize=False)
    pred_proba = pred
    print("Accuracy: ", accuracy / float(pred.size))

    for i in xrange(0,pred.size):
        if (pred[i] >= 0.5):
            pred[i] = 1
        else: 
            pred[i] = 0

    print('---Classification Report---')
    print(classification_report(y_test,pred))
    print('AUROC :' + str(roc_auc_score(y_test, pred_proba)))
    
    
    

In [173]:
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=18,
 seed=27)
modelfit(xgb1)

('Accuracy: ', 0.8879732165851146)
---Classification Report---
             precision    recall  f1-score   support

          0       0.96      0.92      0.94      3685
          1       0.16      0.28      0.20       198

avg / total       0.92      0.89      0.90      3883

AUROC :0.6016583747927031


In [23]:
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}

gsearch1 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=18, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch1.fit(X_train, y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/home/gonzalodiz/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.82340, std: 0.00867, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.82474, std: 0.00837, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.82608, std: 0.00986, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.80007, std: 0.01205, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.80125, std: 0.01319, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.80381, std: 0.01322, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.77699, std: 0.01427, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.77662, std: 0.01608, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.78440, std: 0.01697, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.75554, std: 0.01537, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.75762, std: 0.01802, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.76888, std: 0.01677, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 5

In [27]:
param_test2 = {
    'max_depth':[1,2,3],
    'min_child_weight':[4,5,6]
}

gsearch2 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=18, seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch2.fit(X_train, y_train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

/home/gonzalodiz/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.83085, std: 0.01097, params: {'max_depth': 1, 'min_child_weight': 4},
  mean: 0.83080, std: 0.01090, params: {'max_depth': 1, 'min_child_weight': 5},
  mean: 0.83081, std: 0.01092, params: {'max_depth': 1, 'min_child_weight': 6},
  mean: 0.83077, std: 0.01075, params: {'max_depth': 2, 'min_child_weight': 4},
  mean: 0.83120, std: 0.01159, params: {'max_depth': 2, 'min_child_weight': 5},
  mean: 0.83076, std: 0.01113, params: {'max_depth': 2, 'min_child_weight': 6},
  mean: 0.82580, std: 0.00933, params: {'max_depth': 3, 'min_child_weight': 4},
  mean: 0.82608, std: 0.00986, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.82481, std: 0.01007, params: {'max_depth': 3, 'min_child_weight': 6}],
 {'max_depth': 2, 'min_child_weight': 5},
 0.8312013260081222)

In [87]:
param_test2b = {
    'min_child_weight':[4,5,6]
}

gsearch2 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=3,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=25, seed=27), 
 param_grid = param_test2b, scoring='f1',n_jobs=4,iid=False, cv=5)

gsearch2.fit(X_train, y_train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: 0.22617, std: 0.01726, params: {'min_child_weight': 4},
  mean: 0.22624, std: 0.01454, params: {'min_child_weight': 5},
  mean: 0.22535, std: 0.01492, params: {'min_child_weight': 6}],
 {'min_child_weight': 5},
 0.2262385879618499)

In [28]:
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=2,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=18,
 seed=27)
modelfit(xgb1)

('Accuracy: ', 0.7828998197270152)
---Classification Report---
             precision    recall  f1-score   support

          0       0.98      0.78      0.87      3693
          1       0.15      0.75      0.25       190

avg / total       0.94      0.78      0.84      3883

AUROC :0.7660481422891103


In [29]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train,y_train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

/home/gonzalodiz/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.83120, std: 0.01159, params: {'gamma': 0.0},
  mean: 0.83120, std: 0.01159, params: {'gamma': 0.1},
  mean: 0.83120, std: 0.01159, params: {'gamma': 0.2},
  mean: 0.83120, std: 0.01159, params: {'gamma': 0.3},
  mean: 0.83120, std: 0.01159, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 0.8312013260081222)

In [30]:
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=2,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=18,
 seed=27)
modelfit(xgb1)

('Accuracy: ', 0.7828998197270152)
---Classification Report---
             precision    recall  f1-score   support

          0       0.98      0.78      0.87      3693
          1       0.15      0.75      0.25       190

avg / total       0.94      0.78      0.84      3883

AUROC :0.7660481422891103


In [31]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}

gsearch4 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train,y_train)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

/home/gonzalodiz/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.83152, std: 0.01084, params: {'subsample': 0.6, 'colsample_bytree': 0.6},
  mean: 0.83127, std: 0.01021, params: {'subsample': 0.7, 'colsample_bytree': 0.6},
  mean: 0.83172, std: 0.01065, params: {'subsample': 0.8, 'colsample_bytree': 0.6},
  mean: 0.83103, std: 0.00936, params: {'subsample': 0.9, 'colsample_bytree': 0.6},
  mean: 0.83021, std: 0.01188, params: {'subsample': 0.6, 'colsample_bytree': 0.7},
  mean: 0.83121, std: 0.01101, params: {'subsample': 0.7, 'colsample_bytree': 0.7},
  mean: 0.83199, std: 0.01124, params: {'subsample': 0.8, 'colsample_bytree': 0.7},
  mean: 0.83198, std: 0.01026, params: {'subsample': 0.9, 'colsample_bytree': 0.7},
  mean: 0.82918, std: 0.01267, params: {'subsample': 0.6, 'colsample_bytree': 0.8},
  mean: 0.83021, std: 0.01191, params: {'subsample': 0.7, 'colsample_bytree': 0.8},
  mean: 0.83120, std: 0.01159, params: {'subsample': 0.8, 'colsample_bytree': 0.8},
  mean: 0.83104, std: 0.01014, params: {'subsample': 0.9, 'colsample_bytree'

In [33]:
param_test5 = {
    'subsample':[i/100.0 for i in range(70,90,5)],
    'colsample_bytree':[i/100.0 for i in range(40,70,5)]
}

gsearch5 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(X_train,y_train)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

/home/gonzalodiz/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.83225, std: 0.00973, params: {'subsample': 0.7, 'colsample_bytree': 0.4},
  mean: 0.83087, std: 0.00947, params: {'subsample': 0.75, 'colsample_bytree': 0.4},
  mean: 0.83131, std: 0.00947, params: {'subsample': 0.8, 'colsample_bytree': 0.4},
  mean: 0.83110, std: 0.00865, params: {'subsample': 0.85, 'colsample_bytree': 0.4},
  mean: 0.83225, std: 0.00973, params: {'subsample': 0.7, 'colsample_bytree': 0.45},
  mean: 0.83087, std: 0.00947, params: {'subsample': 0.75, 'colsample_bytree': 0.45},
  mean: 0.83131, std: 0.00947, params: {'subsample': 0.8, 'colsample_bytree': 0.45},
  mean: 0.83110, std: 0.00865, params: {'subsample': 0.85, 'colsample_bytree': 0.45},
  mean: 0.83127, std: 0.01021, params: {'subsample': 0.7, 'colsample_bytree': 0.5},
  mean: 0.83117, std: 0.01012, params: {'subsample': 0.75, 'colsample_bytree': 0.5},
  mean: 0.83172, std: 0.01065, params: {'subsample': 0.8, 'colsample_bytree': 0.5},
  mean: 0.83273, std: 0.00992, params: {'subsample': 0.85, 'colsamp

In [41]:
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=2,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.9,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=18,
 seed=27)
modelfit(xgb1)

('Accuracy: ', 0.7865052794231264)
---Classification Report---
             precision    recall  f1-score   support

          0       0.98      0.79      0.88      3693
          1       0.15      0.75      0.26       190

avg / total       0.94      0.79      0.85      3883

AUROC :0.7679436202203316


In [42]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}

gsearch6 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.9,
objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(X_train,y_train)
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

/home/gonzalodiz/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.82993, std: 0.00955, params: {'reg_alpha': 1e-05},
  mean: 0.83033, std: 0.00974, params: {'reg_alpha': 0.01},
  mean: 0.82997, std: 0.00976, params: {'reg_alpha': 0.1},
  mean: 0.82967, std: 0.00999, params: {'reg_alpha': 1},
  mean: 0.83323, std: 0.00994, params: {'reg_alpha': 100}],
 {'reg_alpha': 100},
 0.8332269153677311)

In [45]:
param_test7 = {
    'reg_alpha':[100,120]
}

gsearch7 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.9,
objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch7.fit(X_train,y_train)
gsearch7.grid_scores_, gsearch7.best_params_, gsearch6.best_score_

/home/gonzalodiz/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.83323, std: 0.00994, params: {'reg_alpha': 100},
  mean: 0.83352, std: 0.01065, params: {'reg_alpha': 120}],
 {'reg_alpha': 120},
 0.8332269153677311)

In [46]:
param_test8 = {
    'learning_rate':[0, 0.001, 0.01, 0.1, 0.2]
}

gsearch8 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=2,
min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.9, reg_alpha=100,
objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
param_grid = param_test8, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch8.fit(X_train,y_train)
gsearch8.grid_scores_, gsearch8.best_params_, gsearch8.best_score_

/home/gonzalodiz/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.50000, std: 0.00000, params: {'learning_rate': 0},
  mean: 0.81517, std: 0.00731, params: {'learning_rate': 0.001},
  mean: 0.82684, std: 0.01067, params: {'learning_rate': 0.01},
  mean: 0.83323, std: 0.00994, params: {'learning_rate': 0.1},
  mean: 0.83217, std: 0.01041, params: {'learning_rate': 0.2}],
 {'learning_rate': 0.1},
 0.8332269153677311)

In [ ]:
param_test8 = {
    'learning_rate':[0, 0.0001, 0.001]
}

gsearch8 = GridSearch(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=7,
min_child_weight=3, gamma=0.3, subsample=0.6, colsample_bytree=0.45,
objective= 'binary:logistic', nthread=4, scale_pos_weight=18,seed=27), 
param_grid = param_test8, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch8.fit(X_train,y_train)
gsearch8.grid_scores_, gsearch8.best_params_, gsearch8.best_score_

In [167]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20)

In [172]:
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=18,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.9,
 reg_alpha=100,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=18,
 seed=27)
modelfit(xgb1)

('Accuracy: ', 0.7414370332217358)
---Classification Report---
             precision    recall  f1-score   support

          0       0.98      0.74      0.84      3685
          1       0.14      0.77      0.23       198

avg / total       0.94      0.74      0.81      3883

AUROC :0.7538519523594149


In [177]:
xgb1.fit(x,y, eval_metric='aucpr')
pred = (xgb1.predict_proba(z)[:,1])

In [178]:
persons = test_set['person']
submit = pd.DataFrame({'label': pred, 'person': persons})
submit = submit[['person', 'label']]
submit.to_csv('../submits/xgboost.csv', index=False)
no_conversiones = submit.loc[submit['label'] < 0.5]['label'].count()
print 'Cantidad de no conversiones: ' + str(no_conversiones)

conversiones = submit.loc[submit['label'] >= 0.5]['label'].count()
print 'Cantidad de conversiones: ' + str(conversiones)

Cantidad de no conversiones: 17337
Cantidad de conversiones: 2078


In [179]:
submit['label'].mean()

0.14328482747077942

## RANDOM FOREST

In [283]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 100, )
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [284]:
pred = clf.predict_proba(X_test)[:,1]

In [ ]:
cantidad_unos = 0
cantidad_ceros = 0
for i in xrange(0,pred.size):
        if (pred[i] >= 0.5):
            pred[i] = 1
            cantidad_unos += 1
        else: 
            pred[i] = 0
            cantidad_ceros += 1
            
print (accuracy_score(y_test, pred, ))          
print (classification_report(y_test, pred))
print ('Cantidad de 0 :' + str(cantidad_ceros))
print ('Cantidad de 1 :' + str(cantidad_unos))

In [ ]:
pred

In [ ]:
roc_auc_score(y_test,pred)

In [160]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
class_weight=[{0:1, 1:25}]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'class_weight': class_weight
              }
print(random_grid)

{'bootstrap': [True, False], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'class_weight': [{0: 1, 1: 25}]}


In [166]:
crf = RandomForestClassifier()
crf_random = RandomizedSearchCV(estimator= crf, param_distributions= random_grid,
                               n_iter = 100, cv = 3, verbose = 2, random_state=42, n_jobs=-1, scoring='roc_auc')

In [167]:
crf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=30, class_weight={0: 1, 1: 25} 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=30, class_weight={0: 1, 1: 25} 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=30, class_weight={0: 1, 1: 25} 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=2000, min_samples_split=5, max_features=sqrt, max_depth=10, class_weight={0: 1, 1: 25} 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=2000, min_samples_split=5, max_features=sqrt, max_depth=10, class_weight={0: 1, 1: 25} 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=2000, min_samples_split=5, max_features=sqrt, max_depth=10, class_weight={0: 1, 1: 25} 
[CV] bootstrap=False, min_samples_leaf=2, n_estimators=1200, min_samples_split=5, max_fe

[CV] bootstrap=True, min_samples_leaf=4, n_estimators=400, min_samples_split=10, max_features=auto, max_depth=70, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=True, min_samples_leaf=4, n_estimators=400, min_samples_split=10, max_features=auto, max_depth=70, class_weight={0: 1, 1: 25}, total=   5.7s
[CV] bootstrap=True, min_samples_leaf=4, n_estimators=400, min_samples_split=10, max_features=auto, max_depth=70, class_weight={0: 1, 1: 25} 


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.3min


[CV]  bootstrap=True, min_samples_leaf=4, n_estimators=400, min_samples_split=10, max_features=auto, max_depth=70, class_weight={0: 1, 1: 25}, total=   5.6s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=800, min_samples_split=5, max_features=sqrt, max_depth=90, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=True, min_samples_leaf=4, n_estimators=400, min_samples_split=10, max_features=auto, max_depth=70, class_weight={0: 1, 1: 25}, total=   5.7s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=800, min_samples_split=5, max_features=sqrt, max_depth=90, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=1000, min_samples_split=2, max_features=auto, max_depth=50, class_weight={0: 1, 1: 25}, total=  21.7s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=800, min_samples_split=5, max_features=sqrt, max_depth=90, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=1000, min_samples_split=2, max_features=auto, 

[CV] bootstrap=False, min_samples_leaf=1, n_estimators=1800, min_samples_split=10, max_features=sqrt, max_depth=30, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=600, min_samples_split=5, max_features=auto, max_depth=80, class_weight={0: 1, 1: 25}, total=   9.1s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=1800, min_samples_split=10, max_features=sqrt, max_depth=30, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=1800, min_samples_split=2, max_features=auto, max_depth=50, class_weight={0: 1, 1: 25}, total=  36.1s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=1800, min_samples_split=10, max_features=sqrt, max_depth=30, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=600, min_samples_split=5, max_features=auto, max_depth=80, class_weight={0: 1, 1: 25}, total=   9.2s
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=1600, min_samples_split=5, max_features=sqr

...

[CV] bootstrap=True, min_samples_leaf=2, n_estimators=1000, min_samples_split=5, max_features=sqrt, max_depth=10, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=True, min_samples_leaf=2, n_estimators=1000, min_samples_split=5, max_features=sqrt, max_depth=10, class_weight={0: 1, 1: 25}, total=  13.3s
[CV] bootstrap=True, min_samples_leaf=4, n_estimators=200, min_samples_split=5, max_features=auto, max_depth=80, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=True, min_samples_leaf=2, n_estimators=1000, min_samples_split=5, max_features=sqrt, max_depth=10, class_weight={0: 1, 1: 25}, total=  13.1s
[CV] bootstrap=True, min_samples_leaf=4, n_estimators=200, min_samples_split=5, max_features=auto, max_depth=80, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=1600, min_samples_split=10, max_features=sqrt, max_depth=None, class_weight={0: 1, 1: 25}, total=  24.1s
[CV] bootstrap=True, min_samples_leaf=4, n_estimators=200, min_samples_split=5, max_features=auto, max

[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=100, class_weight={0: 1, 1: 25}, total=   6.1s
[CV] bootstrap=True, min_samples_leaf=4, n_estimators=800, min_samples_split=10, max_features=sqrt, max_depth=100, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=sqrt, max_depth=100, class_weight={0: 1, 1: 25}, total=   6.1s
[CV] bootstrap=True, min_samples_leaf=4, n_estimators=800, min_samples_split=10, max_features=sqrt, max_depth=100, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=800, min_samples_split=5, max_features=sqrt, max_depth=70, class_weight={0: 1, 1: 25}, total=  16.0s
[CV] bootstrap=True, min_samples_leaf=4, n_estimators=800, min_samples_split=10, max_features=sqrt, max_depth=100, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=600, min_samples_split=10, max_features=sqrt

[CV] bootstrap=False, min_samples_leaf=4, n_estimators=200, min_samples_split=5, max_features=auto, max_depth=100, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=False, min_samples_leaf=2, n_estimators=400, min_samples_split=10, max_features=auto, max_depth=40, class_weight={0: 1, 1: 25}, total=   8.3s
[CV] bootstrap=False, min_samples_leaf=4, n_estimators=200, min_samples_split=5, max_features=auto, max_depth=100, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=200, min_samples_split=5, max_features=auto, max_depth=100, class_weight={0: 1, 1: 25}, total=   4.0s
[CV] bootstrap=False, min_samples_leaf=4, n_estimators=200, min_samples_split=5, max_features=auto, max_depth=100, class_weight={0: 1, 1: 25} 
[CV]  bootstrap=False, min_samples_leaf=4, n_estimators=200, min_samples_split=5, max_features=auto, max_depth=100, class_weight={0: 1, 1: 25}, total=   3.9s
[CV] bootstrap=True, min_samples_leaf=2, n_estimators=2000, min_samples_split=2, max_features=sqr

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 13.7min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'bootstrap': [True, False], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'class_weight': [{0: 1, 1: 25}]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='roc_auc', 

In [168]:
crf_random.best_params_

#{'bootstrap': False,
# 'max_depth': 10,
# 'max_features': 'auto',
# 'min_samples_leaf': 4,
# 'min_samples_split': 5,
# 'n_estimators': 1800}

{'bootstrap': True,
 'class_weight': {0: 1, 1: 25},
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 400}

In [285]:
clf = RandomForestClassifier(n_estimators = 400, max_depth = 10,
                            max_features='sqrt',
                            min_samples_leaf=4,
                            min_samples_split=2,
                            bootstrap=True,
                            class_weight={0:1, 1:25}
                        )
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 25},
            criterion='gini', max_depth=10, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=4,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=400, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [286]:
pred = clf.predict_proba(X_test)[:,1]
cantidad_unos = 0
cantidad_ceros = 0
for i in xrange(0,pred.size):
        if (pred[i] >= 0.5):
            pred[i] = 1
            cantidad_unos += 1
        else: 
            pred[i] = 0
            cantidad_ceros += 1
            
print (accuracy_score(y_test, pred, ))          
print (classification_report(y_test, pred))
print ('Cantidad de 0 :' + str(cantidad_ceros))
print ('Cantidad de 1 :' + str(cantidad_unos))
print ('AUC ROC ' + str(roc_auc_score(y_test, pred)))


0.8441926345609065
             precision    recall  f1-score   support

          0       0.98      0.86      0.91      3684
          1       0.19      0.64      0.30       199

avg / total       0.94      0.84      0.88      3883

Cantidad de 0 :3221
Cantidad de 1 :662
AUC ROC 0.7491324701684317


In [177]:
clf.fit(x,y)
pred = (clf.predict_proba(z)[:,1])

In [287]:
persons = test_set['person']
submit = pd.DataFrame({'label': pred, 'person': persons})
submit = submit[['person', 'label']]
submit.to_csv('../submits/random_forest.csv', index=False)
no_conversiones = submit.loc[submit['label'] < 0.5]['label'].count()
print 'Cantidad de no conversiones: ' + str(no_conversiones)

conversiones = submit.loc[submit['label'] >= 0.5]['label'].count()
print 'Cantidad de conversiones: ' + str(conversiones)

ValueError: array length 3883 does not match index length 19415

In [179]:
from sklearn.ensemble import VotingClassifier

In [288]:
classifiers = []
classifiers.append(('xgb',xgb1))
classifiers.append(('rf',clf))

In [289]:
vc = VotingClassifier(classifiers, voting='soft', n_jobs=-1,
                    weights=[20,1])

In [290]:
vc.fit(X_train, y_train)

VotingClassifier(estimators=[('xgb', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=1, min_child_weight=3, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='binary:logistic', random...mators=400, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=-1, voting='soft', weights=[20, 1])

In [294]:
pred = vc.predict_proba(X_test)[:,1]

In [295]:
cantidad_unos = 0
cantidad_ceros = 0
for i in xrange(0,pred.size):
        if (pred[i] >= 0.5):
            pred[i] = 1
            cantidad_unos += 1
        else: 
            pred[i] = 0
            cantidad_ceros += 1
            
print (accuracy_score(y_test, pred, ))          
print (classification_report(y_test, pred))
print ('Cantidad de 0 :' + str(cantidad_ceros))
print ('Cantidad de 1 :' + str(cantidad_unos))
print ('AUC ROC ' + str(roc_auc_score(y_test, pred)))

0.730620654133402
             precision    recall  f1-score   support

          0       0.98      0.73      0.84      3684
          1       0.13      0.77      0.23       199

avg / total       0.94      0.73      0.81      3883

Cantidad de 0 :2730
Cantidad de 1 :1153
AUC ROC 0.7487000692932634


In [263]:
vc.fit(x,y)
pred = (vc.predict_proba(z)[:,1])

In [264]:
persons = test_set['person']
submit = pd.DataFrame({'label': pred, 'person': persons})
submit = submit[['person', 'label']]
submit.to_csv('../submits/vc_xgb_rf.csv', index=False)
no_conversiones = submit.loc[submit['label'] < 0.5]['label'].count()
print 'Cantidad de no conversiones: ' + str(no_conversiones)

conversiones = submit.loc[submit['label'] >= 0.5]['label'].count()
print 'Cantidad de conversiones: ' + str(conversiones)

Cantidad de no conversiones: 13210
Cantidad de conversiones: 6205
